In [115]:
import os
import glob
import numpy as np
import pandas as pd
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry, ApertureStats

In [116]:
def load_fits_images(dir: str, observation_date: str) -> tuple:
    """
    Carrega as imagens FITS de um diretório específico, excluindo imagens com nuvens.

    Parâmetros:
    dir (str): O diretório onde as imagens FITS estão armazenadas.
    observation_date (str): A data da observação para criar o caminho do diretório.

    Retorna:
    tuple: Uma tupla contendo:
        - image_names (list): Lista dos nomes das imagens carregadas.
        - images_data (list): Lista dos dados das imagens carregadas.
    """
    base_path = os.path.join(dir, f'{observation_date}/') # Constrói o caminho base para o diretório das imagens, combinando o diretório raiz e a data de observação
    file_pattern = os.path.join(base_path, '*.fit') # Define o padrão para localizar os arquivos FITS (qualquer arquivo com extensão .fit no diretório)
    fits_files = glob.glob(file_pattern) # Usa glob para encontrar todos os arquivos FITS que correspondem ao padrão especificado
    
    # Filtra a lista de arquivos para excluir aqueles cujo nome esteja no intervalo 'pg1530-021.fit' a 'pg1530-050.fit'
    # Essas imagens contêm nuvens ou outros problemas, por isso são excluídas
    image_names = sorted([f for f in fits_files if not ('pg1530-021.fit' <= os.path.basename(f) <= 'pg1530-050.fit')])
    
    images_data = [] # Inicializa uma lista para armazenar os dados das imagens
    for fits_file in image_names: # Itera sobre os arquivos FITS filtrados
        hdu = fits.open(fits_file) # Abre o arquivo FITS e carrega o primeiro HDU (Header Data Unit), que contém os dados da imagem
        images_data.append(hdu[0].data) # Adiciona os dados da imagem à lista images_data
    
    return image_names, images_data # Retorna duas listas: uma com os nomes dos arquivos e outra com os dados das imagens

In [117]:
def calculate_photometry(dir: str, observation_date: str, stars_info: list[tuple], radius_in: float, dannulus: float, radius_aperture: float) -> tuple:
    """
    Calcula a fotometria de abertura para as imagens carregadas.

    Parâmetros:
    dir (str): O diretório onde as imagens FITS estão armazenadas.
    observation_date (str): A data da observação para a qual as imagens serão processadas.
    stars_info (list): Lista de informações (id, x_pos, y_pos, x_crop, y_crop, grau_rot) sobre as estrelas a serem analisadas,
                       onde cada estrela é representada como uma tupla.
    radius_in (float): O raio interno do anel circular utilizado para calcular o fundo.
    dannulus (float): A largura do anel de fundo (ou seja, a largura do anel externo).
    radius_aperture (float): O raio da abertura circular a ser usada para a fotometria.

    Retorna:
    results (list): Uma lista de resultados, onde cada resultado é uma lista contendo:
        - image_name (str): Nome do arquivo da imagem.
        - image_data (np.ndarray): Dados da imagem.
        - rotate_grau (float): O grau de rotação aplicado à seção da imagem.
        - aperture_cropped (CircularAperture): A abertura circular criada para a fotometria.
        - x_star (float): A posição x da estrela na imagem.
        - y_star (float): A posição y da estrela na imagem.
        - annulus (CircularAnnulus): O anel circular criado para calcular o fundo.
        - dannulus (float): A largura do anel de fundo.
        - mag (float): A magnitude da estrela calculada.
        - MERR (float): O erro associado à magnitude.
    """
    # Carrega as imagens FITS do diretório especificado para a data da observação
    image_names, images_data = load_fits_images(dir=dir, observation_date=observation_date)
    
    results = [] # Inicializa uma lista para armazenar os resultados da fotometria
    # Itera sobre as informações das estrelas, os dados das imagens e os nomes das imagens
    for star_info, image_data, image_name in zip(stars_info, images_data, image_names):
        _, x_star, y_star, row_slice, col_slice, rotate_grau = star_info  # Desempacota as informações da estrela
        
        # Obtém a seção da imagem correspondente à posição da estrela usando os slices
        image_section = image_data[row_slice, col_slice]
        if rotate_grau!=0: # Rotaciona a seção da imagem se necessário
            image_section = rotate(image_section, rotate_grau, reshape=False)
        
        # Cria uma abertura circular para a fotometria com a posição da estrela
        aperture_cropped = CircularAperture((x_star, y_star), r=radius_aperture)

        # Cria um anel circular para calcular o fundo, com raio interno e externo definidos
        annulus = CircularAnnulus((x_star, y_star), r_in=radius_in, r_out=radius_in + dannulus)
        
        # Calcula a fotometria de abertura usando a seção da imagem e a abertura criada
        photometry = aperture_photometry(image_section, aperture_cropped)

        # Calcula a contribuição do céu usando o anel circular
        bkg = ApertureStats(image_section, annulus) # Estatísticas do fundo
        bkg_sum = bkg.sum # Soma dos pixels no anel de fundo
        bkg_mean = bkg.mean # Média dos valores de pixel no anel de fundo
        total_bkg = bkg_mean * aperture_cropped.area # Contribuição total do fundo

        star_flux = photometry["aperture_sum"] - total_bkg # Calcula o fluxo da estrela subtraindo o fundo
        FERR = np.sqrt(photometry['aperture_sum'][0]) + np.sqrt(bkg_sum) # Calcula o erro associado ao fluxo total

        zmag = 25  # Valor de referência (IRAF)
        mag = zmag - 2.5 * np.log10(star_flux) if star_flux > 0 else np.inf # Calcula a magnitude da estrela usando a fórmula padrão

        if star_flux > 0: # Calcula o erro da magnitude (MERR) se o fluxo for maior que zero
            MERR = (1.0857 * FERR) / star_flux[0]
        else:
            MERR = np.inf  # Erro infinito se o fluxo total for zero

        # Adiciona os resultados da fotometria para esta estrela na lista de resultados
        results.append([image_name.split("/")[2],  # Nome do arquivo da imagem
                        image_section,    # Seção da imagem
                        rotate_grau,      # Grau de rotação aplicado
                        aperture_cropped, # Abertura circular
                        x_star,           # Posição x da estrela
                        y_star,           # Posição y da estrela
                        annulus,          # Anel circular
                        dannulus,         # Largura do anel de fundo
                        mag.item() if isinstance(mag, np.ndarray) else mag, # Magnitude
                        float(MERR)])     # Erro da magnitude
    
    # Criar um DataFrame do pandas a partir dos resultados
    df = pd.DataFrame(results, columns=['image_name', 'image_section', 'rotate_grau', 'aperture_cropped' ,'x_star', 'y_star','annulus','dannulus', 'mag', 'MERR'])
    
    # (Opcional) Salvar os resultados em arquivos CSV (comentado)
    #df[['image_name', 'mag', 'MERR','rotate_grau','x_star', 'y_star']].to_csv(f"photometry_results.csv", index=False, sep=";")
    #df[['image_name', 'mag', 'MERR']].to_csv(f"{dir}/{observation_date}/results/photometry_results.csv", index=False)

    return results # Retorna a lista de resultados da fotometria

In [136]:
def plot_images(photometry_results,dir ,observation_date , rows: int = 4, columns: int = 5) -> None:
    """
    Plota as imagens com fotometria e anotações, exibindo as magnitudes e erros associados a cada estrela.

    Parâmetros:
    photometry_results (list): Lista contendo os resultados da fotometria, onde cada resultado é uma tupla com:
        - image_name (str): Nome do arquivo da imagem.
        - image_section (np.ndarray): Dados da seção da imagem a ser plotada.
        - rotate_grau (float): Grau de rotação aplicado à imagem.
        - aperture (CircularAperture): Abertura circular utilizada para a fotometria.
        - x_star (float): Posição x da estrela na imagem.
        - y_star (float): Posição y da estrela na imagem.
        - annulus (CircularAnnulus): Anel circular usado para calcular o fundo.
        - dannulus (float): Largura do anel de fundo.
        - mag (float): Magnitude da estrela calculada.
        - MERR (float): Erro associado à magnitude.

    dir (str): O diretório onde as imagens ou resultados podem ser salvos.

    observation_date (str): A data da observação, usada no título do gráfico.

    rows (int, opcional): O número de linhas para a exibição das imagens. O padrão é 4.

    columns (int, opcional): O número de colunas para a exibição das imagens. O padrão é 5.

    Retorna:
    None: A função apenas gera um gráfico e não retorna valores.
    """

    fig, axes = plt.subplots(rows, columns, figsize=(20, 10)) # Cria uma figura com subplots para exibir as imagens
    fig.suptitle(f"{observation_date} PG1530+057A", fontsize=20, y=1) # Define o título da figura usando a data da observação
    
    ims = [] # Lista para armazenar os objetos de imagem para a colorbar
    # Itera sobre os resultados da fotometria para plotar cada imagem
    for i, (image_name, image_section, rotate_grau, aperture,_,_, annulus, _, mag, MERR) in enumerate(photometry_results):
        # Cria uma string para mostrar informações sobre a magnitude e o erro
        textstr = '\n'.join((
            f"MAG:       {mag[0]:.3f}" if isinstance(mag, np.ndarray) else f"MAG:       {mag:.3f}",
            f"MERR:      {MERR:.3f}",
        ))

        # Define os limites de exibição para a imagem (percentis para escala de intensidade)
        vmin = np.percentile(image_section, 1)
        vmax = np.percentile(image_section, 99)

        # Acessa o eixo correspondente na grade de subplots
        ax = axes[i // columns, i % columns]
        im = ax.imshow(image_section, cmap='gray', origin='lower', vmin=vmin, vmax=vmax) #Plota a imagem
        ims.append(im) # Adiciona o objeto de imagem à lista
        aperture.plot(color='red', lw=1.5, ax=ax, label='Aperture') # Plota a abertura circular na imagem
        annulus.plot(color='blue', lw=1.5, ax=ax, label='Annulus')  # Plota o anel circular na imagem       

        # Define a posição do texto a ser adicionado na imagem
        x_text = 0.05 * image_section.shape[1]
        y_text = 0.95 * image_section.shape[0]
        # Adiciona o texto informativo na imagem
        ax.text(x_text, y_text, textstr, fontsize=10, color='white', ha='left', va='top',
                bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor='black', alpha=0.8))
        ax.set_title(image_name.split("/")[-1], fontsize=10, pad=10) # Define o título do subplot como o nome do arquivo da imagem

        # Plotar o anel
        annulus.plot(color='blue', lw=1.5, ax=ax, label='Annulus')

        # Cria uma colorbar única para todos os subplots, posicionada na parte inferior
        #cbar_ax = fig.add_axes([0.15, 0.05, 0.7, 0.02])
        cbar = plt.colorbar(ims[i],ax=ax, orientation='vertical') # Usa a primeira imagem para a colorbar
        cbar.ax.tick_params(labelsize=10) # Ajusta o tamanho dos ticks da colorbar

    # Remove os eixos das imagens que não são utilizadas, se houver
    for j in range(len(image_section), rows * columns):
        axes[j // columns, j % columns].axis('off')

    # Ajustar layout para evitar sobreposições
    plt.subplots_adjust(top=0.93, bottom=0.1, left=0.07, right=0.93, hspace=0.5, wspace=0.2)
    
    # (Opcional) Salvar a figura como um arquivo PNG ou PDF (comentado)
    #plt.savefig(f"{dir}{observation_date}/results/aperture_photometry")
    
    plt.show() # Exibe o gráfico

In [137]:
# Configura o backend do matplotlib para exibir gráficos em uma janela separada
%matplotlib qt

dir = "observations/" # Define o diretório onde as observações estão armazenadas
observation_date = "2011-06-26" # Define a data da observação
radius_aperture = 10 # Define o raio da abertura circular para a fotometria
# Define o raio interno e a largura do anel de fundo para a fotometria
radius_in = 13
dannulus = 5

# id, x_pos, y_pos, slice(coord y), slice(coord x), grau de rotacao
stars_info = [(0,  64,     110,     slice(350,700),  slice(450, 910),   0  ),
              (1,  200.4,  186.4,   slice(250,750),  slice(50,  710),   180),
              (2,  91.4,   137.4,   slice(300,700),  slice(100, 600),  -180),
              (3,  92.5,   87,      slice(300,650),  slice(100, 600),  -180),
              (4,  93.20,  87.38,   slice(300,650),  slice(100, 600),  -180),
              (5,  94,     87.5,    slice(300,650),  slice(100, 600),  -180),
              (6,  95,     88.5,    slice(300,650),  slice(200, 600),  -180),
              (7,  95.7,   88.3,    slice(300,650),  slice(200, 600),  -180),
              (8,  97,     89.5,    slice(300,650),  slice(100, 600),  -180),
              (9,  97.8,   90.1,    slice(300,650),  slice(100, 600),  -180),
              (10, 98.5,   95.05,   slice(300,650),  slice(350, 800),  -180),
              (11, 98.6,   95.1,    slice(300,650),  slice(350, 800),  -180),
              (12, 98.6,   95,      slice(300,650),  slice(350, 800),  -180),
              (13, 98.4,   93.8,    slice(300,650),  slice(350, 800),  -180),
              (14, 98.3,   94,      slice(300,650),  slice(350, 800),   180),
              (15, 72.5,   167,     slice(300,700),  slice(250, 700),   0  ),
              (16, 71.5,   166,     slice(300,700),  slice(250, 700),   0  ),
              (17, 72,     165,     slice(300,700),  slice(250, 700),   0  ),
              (18, 72.,    164.2,   slice(300,700),  slice(250, 700),   0  ),
              (19, 72.6,   164.2,   slice(300,700),  slice(250, 700),   0  ),]


# Executa a fotometria de abertura com as informações acima
photometry_results = calculate_photometry(dir=dir, observation_date=observation_date, stars_info=stars_info,
                     radius_in=radius_in, dannulus=dannulus, radius_aperture=radius_aperture)

# Plota as imagens com a fotometria calculada
plot_images(dir=dir, observation_date=observation_date,photometry_results=photometry_results)